In [1]:
import sys
from six import iteritems
a = { i:1 for i in range(10000)}
b = a.items()
c = iteritems(a)
print(sys.getsizeof(a))
print(sys.getsizeof(b))
print(sys.getsizeof(c))
print(c)
print(type(a), type(b), type(c),)

295008
48
80
<class 'dict'> <class 'dict_items'> <class 'dict_itemiterator'>


In [2]:
from gensim import corpora
from smart_open import open # for transparently opening remote files

dictionary = corpora.Dictionary(line.lower().split() for line in open('data/mycorpus.txt'))

stoplist = set('for a of the and to in'.split(' '))
stop_ids = [dictionary.token2id[stopword] for stopword in stoplist
            if stopword in dictionary.token2id]
once_ids = [tokenid for tokenid, docfreq in dictionary.dfs.items() if docfreq == 1]

# 移除stop words,和只出现一次的words
print("stop_ids =", stop_ids)
print("once_ids =", once_ids)
dictionary.filter_tokens(stop_ids + once_ids) 

# remove gaps in id sequence after words that were removed
dictionary.compactify()  
print(dictionary)

stop_ids = [19, 9, 3, 8, 26, 18, 33]
once_ids = [7, 3, 6, 0, 1, 10, 17, 20, 21, 25, 24, 26, 22, 23, 28, 29, 27, 31, 34, 35, 33, 36, 41, 40, 39, 38]
Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...)


In [33]:
print(dictionary.dfs)

{1: 2, 2: 2, 0: 2, 4: 2, 7: 3, 5: 3, 3: 2, 6: 2, 8: 2, 9: 3, 10: 3, 11: 2}


In [12]:
class MyCorpus(object):
    def __init__(self, file_path='data/mycorpus.txt'):
        self.file_path = file_path
    
    def __len__(self):
        with open(self.file_path) as f:
            for i, _ in enumerate(f):
                pass
        return i + 1
    
    def __iter__(self):
        for line in open(self.file_path):
            yield dictionary.doc2bow(line.lower().split())

corpus_memory_friendly = MyCorpus()  # doesn't load the corpus into memory!
print(corpus_memory_friendly)
for vector in corpus_memory_friendly:  # load one vector into memory at a time
    print(vector)  

[(0, 1), (1, 1), (2, 1)]
[(0, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)]
[(2, 1), (5, 1), (7, 1), (8, 1)]
[(1, 1), (5, 2), (8, 1)]
[(3, 1), (6, 1), (7, 1)]
[(9, 1)]
[(9, 1), (10, 1)]
[(9, 1), (10, 1), (11, 1)]
[(4, 1), (10, 1), (11, 1)]
[(0, 1), (1, 1), (2, 1)]
[(0, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)]
[(2, 1), (5, 1), (7, 1), (8, 1)]
[(1, 1), (5, 2), (8, 1)]
[(3, 1), (6, 1), (7, 1)]
[(9, 1)]
[(9, 1), (10, 1)]
[(9, 1), (10, 1), (11, 1)]
[(4, 1), (10, 1), (11, 1)]


In [14]:
corpora.MmCorpus.serialize('/tmp/corpus.mm', corpus_memory_friendly)
corpora.SvmLightCorpus.serialize('/tmp/corpus.svmlight', corpus_memory_friendly)
corpora.BleiCorpus.serialize('/tmp/corpus.lda-c', corpus_memory_friendly)
corpora.LowCorpus.serialize('/tmp/corpus.low', corpus_memory_friendly)


In [25]:
print('-'*50)
corpus = corpora.MmCorpus('/tmp/corpus.mm')
print(corpus)
print(corpus[0])

print('-'*50)
corpus = corpora.SvmLightCorpus('/tmp/corpus.svmlight')
print(corpus)
print(corpus[0])

print('-'*50)
corpus = corpora.BleiCorpus('/tmp/corpus.lda-c')
print(corpus)
print(corpus[0])

print('-'*50)
corpus = corpora.LowCorpus('/tmp/corpus.low')
print(corpus)
print(corpus[0])

print('-'*50)
for doc in corpus:
    print(doc)

--------------------------------------------------
MmCorpus(9 documents, 12 features, 28 non-zero entries)
[(0, 1.0), (1, 1.0), (2, 1.0)]
--------------------------------------------------
[(0, 1.0), (1, 1.0), (2, 1.0)]
--------------------------------------------------
[(0, 1.0), (1, 1.0), (2, 1.0)]
--------------------------------------------------
[(0, 1), (1, 1), (4, 1)]
--------------------------------------------------
[(0, 1), (1, 1), (4, 1)]
[(0, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1)]
[(4, 1), (7, 1), (9, 1), (10, 1)]
[(1, 1), (7, 2), (10, 1)]
[(5, 1), (8, 1), (9, 1)]
[(11, 1)]
[(11, 1), (2, 1)]
[(11, 1), (2, 1), (3, 1)]
[(6, 1), (2, 1), (3, 1)]


In [27]:
import gensim
import numpy as np
import scipy.sparse

np.random.seed(20200609)
scipy_sparse_matrix = scipy.sparse.random(5, 2, density=0.6) 
print("scipy_sparse_matrix =\n", scipy_sparse_matrix)

corpus = gensim.matutils.Sparse2Corpus(scipy_sparse_matrix)
print("corpus =", corpus)
for doc in corpus:
    print(doc)
    
new_matrix = gensim.matutils.corpus2csc(corpus)
print("new_matrix =\n", new_matrix)

scipy_sparse_matrix =
   (4, 0)	0.6501933321119864
  (3, 0)	0.16918575812686232
  (4, 1)	0.9668114763742701
  (1, 1)	0.6894940466416551
  (2, 0)	0.6503421133138086
  (0, 1)	0.8242847957249896
corpus = <gensim.matutils.Sparse2Corpus object at 0x7f3159477710>
[(2, 0.6503421133138086), (3, 0.16918575812686232), (4, 0.6501933321119864)]
[(0, 0.8242847957249896), (1, 0.6894940466416551), (4, 0.9668114763742701)]
new_matrix =
   (2, 0)	0.6503421133138086
  (3, 0)	0.16918575812686232
  (4, 0)	0.6501933321119864
  (0, 1)	0.8242847957249896
  (1, 1)	0.6894940466416551
  (4, 1)	0.9668114763742701


## print(scipy.sparse.random(5, 2))

In [32]:
%matplotlib inline


Corpora and Vector Spaces
=========================

Demonstrates transforming text into a vector space representation.

Also introduces corpus streaming and persistence to disk in various formats.


In [15]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

First, let’s create a small corpus of nine short documents [1]_:


From Strings to Vectors
------------------------

This time, let's start from documents represented as strings:




In [16]:
documents = [
    "Human machine interface for lab abc computer applications",
    "A survey of user opinion of computer system response time",
    "The EPS user interface management system",
    "System and human system engineering testing of EPS",
    "Relation of user perceived response time to error measurement",
    "The generation of random binary unordered trees",
    "The intersection graph of paths in trees",
    "Graph minors IV Widths of trees and well quasi ordering",
    "Graph minors A survey",
]

This is a tiny corpus of nine documents, each consisting of only a single sentence.

First, let's tokenize the documents, remove common words (using a toy stoplist)
as well as words that only appear once in the corpus:



In [17]:
from pprint import pprint  # pretty-printer
from collections import defaultdict

# remove common words and tokenize
stoplist = set('for a of the and to in'.split())
texts = [
    [word for word in document.lower().split() if word not in stoplist]
    for document in documents
]

# remove words that appear only once
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [
    [token for token in text if frequency[token] > 1]
    for text in texts
]

pprint(texts)

[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]


Your way of processing the documents will likely vary; here, I only split on whitespace
to tokenize, followed by lowercasing each word. In fact, I use this particular
(simplistic and inefficient) setup to mimic the experiment done in Deerwester et al.'s
original LSA article [1]_.

The ways to process documents are so varied and application- and language-dependent that I
decided to *not* constrain them by any interface. Instead, a document is represented
by the features extracted from it, not by its "surface" string form: how you get to
the features is up to you. Below I describe one common, general-purpose approach (called
:dfn:`bag-of-words`), but keep in mind that different application domains call for
different features, and, as always, it's `garbage in, garbage out <http://en.wikipedia.org/wiki/Garbage_In,_Garbage_Out>`_...

To convert documents to vectors, we'll use a document representation called
`bag-of-words <http://en.wikipedia.org/wiki/Bag_of_words>`_. In this representation,
each document is represented by one vector where each vector element represents
a question-answer pair, in the style of:

- Question: How many times does the word `system` appear in the document?
- Answer: Once.

It is advantageous to represent the questions only by their (integer) ids. The mapping
between the questions and ids is called a dictionary:



In [18]:
from gensim import corpora
dictionary = corpora.Dictionary(texts)
dictionary.save('/tmp/deerwester.dict')  # store the dictionary, for future reference
print(dictionary)

2020-06-08 03:42:37,016 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-06-08 03:42:37,019 : INFO : built Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...) from 9 documents (total 29 corpus positions)
2020-06-08 03:42:37,020 : INFO : saving Dictionary object under /tmp/deerwester.dict, separately None
2020-06-08 03:42:37,023 : INFO : saved /tmp/deerwester.dict


Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...)


Here we assigned a unique integer id to all words appearing in the corpus with the
:class:`gensim.corpora.dictionary.Dictionary` class. This sweeps across the texts, collecting word counts
and relevant statistics. In the end, we see there are twelve distinct words in the
processed corpus, which means each document will be represented by twelve numbers (ie., by a 12-D vector).
To see the mapping between words and their ids:



In [19]:
print(dictionary.token2id)

{'computer': 0, 'human': 1, 'interface': 2, 'response': 3, 'survey': 4, 'system': 5, 'time': 6, 'user': 7, 'eps': 8, 'trees': 9, 'graph': 10, 'minors': 11}


To actually convert tokenized documents to vectors:



In [20]:
new_doc = "Human computer interaction"
new_vec = dictionary.doc2bow(new_doc.lower().split())
print(new_vec)  # the word "interaction" does not appear in the dictionary and is ignored

[(0, 1), (1, 1)]


The function :func:`doc2bow` simply counts the number of occurrences of
each distinct word, converts the word to its integer word id
and returns the result as a sparse vector. The sparse vector ``[(0, 1), (1, 1)]``
therefore reads: in the document `"Human computer interaction"`, the words `computer`
(id 0) and `human` (id 1) appear once; the other ten dictionary words appear (implicitly) zero times.



In [21]:
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize('/tmp/deerwester.mm', corpus)  # store to disk, for later use
print(corpus)

2020-06-08 03:42:44,046 : INFO : storing corpus in Matrix Market format to /tmp/deerwester.mm
2020-06-08 03:42:44,049 : INFO : saving sparse matrix to /tmp/deerwester.mm
2020-06-08 03:42:44,050 : INFO : PROGRESS: saving document #0
2020-06-08 03:42:44,052 : INFO : saved 9x12 matrix, density=25.926% (28/108)
2020-06-08 03:42:44,053 : INFO : saving MmCorpus index to /tmp/deerwester.mm.index


[[(0, 1), (1, 1), (2, 1)], [(0, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)], [(2, 1), (5, 1), (7, 1), (8, 1)], [(1, 1), (5, 2), (8, 1)], [(3, 1), (6, 1), (7, 1)], [(9, 1)], [(9, 1), (10, 1)], [(9, 1), (10, 1), (11, 1)], [(4, 1), (10, 1), (11, 1)]]


In [27]:
corpus1 = corpora.MmCorpus('/tmp/deerwester.mm')
print(corpus1)
print(corpus1)
[item for item in corpus1]

2020-06-08 03:54:48,171 : INFO : loaded corpus index from /tmp/deerwester.mm.index
2020-06-08 03:54:48,172 : INFO : initializing cython corpus reader from /tmp/deerwester.mm
2020-06-08 03:54:48,173 : INFO : accepted corpus with 9 documents, 12 features, 28 non-zero entries


MmCorpus(9 documents, 12 features, 28 non-zero entries)
MmCorpus(9 documents, 12 features, 28 non-zero entries)


[[(0, 1.0), (1, 1.0), (2, 1.0)],
 [(0, 1.0), (3, 1.0), (4, 1.0), (5, 1.0), (6, 1.0), (7, 1.0)],
 [(2, 1.0), (5, 1.0), (7, 1.0), (8, 1.0)],
 [(1, 1.0), (5, 2.0), (8, 1.0)],
 [(3, 1.0), (6, 1.0), (7, 1.0)],
 [(9, 1.0)],
 [(9, 1.0), (10, 1.0)],
 [(9, 1.0), (10, 1.0), (11, 1.0)],
 [(4, 1.0), (10, 1.0), (11, 1.0)]]

By now it should be clear that the vector feature with ``id=10`` stands for the question "How many
times does the word `graph` appear in the document?" and that the answer is "zero" for
the first six documents and "one" for the remaining three.


Corpus Streaming -- One Document at a Time
-------------------------------------------

Note that `corpus` above resides fully in memory, as a plain Python list.
In this simple example, it doesn't matter much, but just to make things clear,
let's assume there are millions of documents in the corpus. Storing all of them in RAM won't do.
Instead, let's assume the documents are stored in a file on disk, one document per line. Gensim
only requires that a corpus must be able to return one document vector at a time:




In [11]:
from smart_open import open  # for transparently opening remote files


class MyCorpus(object):
    def __iter__(self):
        for line in open('https://radimrehurek.com/gensim/mycorpus.txt'):
            # assume there's one document per line, tokens separated by whitespace
            yield dictionary.doc2bow(line.lower().split())

The full power of Gensim comes from the fact that a corpus doesn't have to be
a ``list``, or a ``NumPy`` array, or a ``Pandas`` dataframe, or whatever.
Gensim *accepts any object that, when iterated over, successively yields
documents*.



In [12]:
# This flexibility allows you to create your own corpus classes that stream the
# documents directly from disk, network, database, dataframes... The models
# in Gensim are implemented such that they don't require all vectors to reside
# in RAM at once. You can even create the documents on the fly!

Download the sample `mycorpus.txt file here <./mycorpus.txt>`_. The assumption that
each document occupies one line in a single file is not important; you can mold
the `__iter__` function to fit your input format, whatever it is.
Walking directories, parsing XML, accessing the network...
Just parse your input to retrieve a clean list of tokens in each document,
then convert the tokens via a dictionary to their ids and yield the resulting sparse vector inside `__iter__`.



In [13]:
corpus_memory_friendly = MyCorpus()  # doesn't load the corpus into memory!
print(corpus_memory_friendly)

Corpus is now an object. We didn't define any way to print it, so `print` just outputs address
of the object in memory. Not very useful. To see the constituent vectors, let's
iterate over the corpus and print each document vector (one at a time):



In [14]:
for vector in corpus_memory_friendly:  # load one vector into memory at a time
    print(vector)

HTTPError: 404 Client Error: Not Found for url: https://radimrehurek.com/gensim/mycorpus.txt

Although the output is the same as for the plain Python list, the corpus is now much
more memory friendly, because at most one vector resides in RAM at a time. Your
corpus can now be as large as you want.

Similarly, to construct the dictionary without loading all texts into memory:



In [ ]:
from six import iteritems
# collect statistics about all tokens
dictionary = corpora.Dictionary(line.lower().split() for line in open('https://radimrehurek.com/gensim/mycorpus.txt'))
# remove stop words and words that appear only once
stop_ids = [
    dictionary.token2id[stopword]
    for stopword in stoplist
    if stopword in dictionary.token2id
]
once_ids = [tokenid for tokenid, docfreq in iteritems(dictionary.dfs) if docfreq == 1]
dictionary.filter_tokens(stop_ids + once_ids)  # remove stop words and words that appear only once
dictionary.compactify()  # remove gaps in id sequence after words that were removed
print(dictionary)

And that is all there is to it! At least as far as bag-of-words representation is concerned.
Of course, what we do with such a corpus is another question; it is not at all clear
how counting the frequency of distinct words could be useful. As it turns out, it isn't, and
we will need to apply a transformation on this simple representation first, before
we can use it to compute any meaningful document vs. document similarities.
Transformations are covered in the next tutorial
(`sphx_glr_auto_examples_core_run_topics_and_transformations.py`),
but before that, let's briefly turn our attention to *corpus persistency*.


Corpus Formats
---------------

There exist several file formats for serializing a Vector Space corpus (~sequence of vectors) to disk.
`Gensim` implements them via the *streaming corpus interface* mentioned earlier:
documents are read from (resp. stored to) disk in a lazy fashion, one document at
a time, without the whole corpus being read into main memory at once.

One of the more notable file formats is the `Market Matrix format <http://math.nist.gov/MatrixMarket/formats.html>`_.
To save a corpus in the Matrix Market format:

create a toy corpus of 2 documents, as a plain Python list



In [ ]:
corpus = [[(1, 0.5)], []]  # make one document empty, for the heck of it

corpora.MmCorpus.serialize('/tmp/corpus.mm', corpus)

Other formats include `Joachim's SVMlight format <http://svmlight.joachims.org/>`_,
`Blei's LDA-C format <http://www.cs.princeton.edu/~blei/lda-c/>`_ and
`GibbsLDA++ format <http://gibbslda.sourceforge.net/>`_.



In [ ]:
corpora.SvmLightCorpus.serialize('/tmp/corpus.svmlight', corpus)
corpora.BleiCorpus.serialize('/tmp/corpus.lda-c', corpus)
corpora.LowCorpus.serialize('/tmp/corpus.low', corpus)

Conversely, to load a corpus iterator from a Matrix Market file:



In [ ]:
corpus = corpora.MmCorpus('/tmp/corpus.mm')

Corpus objects are streams, so typically you won't be able to print them directly:



In [ ]:
print(corpus)

Instead, to view the contents of a corpus:



In [ ]:
# one way of printing a corpus: load it entirely into memory
print(list(corpus))  # calling list() will convert any sequence to a plain Python list

or



In [ ]:
# another way of doing it: print one document at a time, making use of the streaming interface
for doc in corpus:
    print(doc)

The second way is obviously more memory-friendly, but for testing and development
purposes, nothing beats the simplicity of calling ``list(corpus)``.

To save the same Matrix Market document stream in Blei's LDA-C format,



In [ ]:
corpora.BleiCorpus.serialize('/tmp/corpus.lda-c', corpus)

In this way, `gensim` can also be used as a memory-efficient **I/O format conversion tool**:
just load a document stream using one format and immediately save it in another format.
Adding new formats is dead easy, check out the `code for the SVMlight corpus
<https://github.com/piskvorky/gensim/blob/develop/gensim/corpora/svmlightcorpus.py>`_ for an example.

Compatibility with NumPy and SciPy
----------------------------------

Gensim also contains `efficient utility functions <http://radimrehurek.com/gensim/matutils.html>`_
to help converting from/to numpy matrices



In [ ]:
import gensim
import numpy as np
numpy_matrix = np.random.randint(10, size=[5, 2])  # random matrix as an example
corpus = gensim.matutils.Dense2Corpus(numpy_matrix)
# numpy_matrix = gensim.matutils.corpus2dense(corpus, num_terms=number_of_corpus_features)

and from/to `scipy.sparse` matrices



In [ ]:
import scipy.sparse
scipy_sparse_matrix = scipy.sparse.random(5, 2)  # random sparse matrix as example
corpus = gensim.matutils.Sparse2Corpus(scipy_sparse_matrix)
scipy_csc_matrix = gensim.matutils.corpus2csc(corpus)

What Next
---------

Read about `sphx_glr_auto_examples_core_run_topics_and_transformations.py`.

References
----------

For a complete reference (Want to prune the dictionary to a smaller size?
Optimize converting between corpora and NumPy/SciPy arrays?), see the `apiref`.

.. [1] This is the same corpus as used in
       `Deerwester et al. (1990): Indexing by Latent Semantic Analysis <http://www.cs.bham.ac.uk/~pxt/IDA/lsa_ind.pdf>`_, Table 2.



Here we show a pretty fastText logo so that our gallery picks it up as a thumbnail.




In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
img = mpimg.imread('run_corpora_and_vector_spaces.png')
imgplot = plt.imshow(img)
plt.axis('off')
plt.show()